In [104]:
import pandas as pd

small = pd.read_csv('hadoop_small_results.csv')


In [107]:
import matplotlib.pyplot as plt
plt.close('all')

perf = small.perf
# perf.plot()
baseline = small.loc[small['config#'] == 0].perf

perf = perf.apply(lambda x: round((baseline-x)*100/baseline, 2))
# perf = perf.loc[perf[0] > 0]
perf = perf.values.flatten().tolist()

print(sorted(perf, reverse=True))



[21.0, 17.08, 7.24, 6.95, 6.4, 5.44, 3.16, 2.23, 1.87, 1.0, 0.43, 0.07, 0.0, -0.37, -0.41, -1.09, -1.81, -2.01, -2.97, -3.29, -3.49, -3.85, -5.25, -5.45, -6.69, -6.77, -7.97, -8.61, -8.85, -9.57, -10.29, -10.34, -11.06, -11.54, -12.57, -12.65, -12.86, -12.89, -13.41, -13.7, -14.1, -14.13, -15.33, -15.57, -16.22, -16.34, -17.42, -17.97, -18.54, -18.74, -20.58, -20.85, -21.22, -24.74, -25.42, -25.47, -25.71, -26.22, -27.9, -28.71, -29.1, -29.14, -32.43, -32.79, -33.91, -34.15, -34.27, -34.78, -35.07, -35.71, -35.86, -36.1, -36.34, -38.35, -38.71, -39.55, -40.35, -40.87, -41.55, -41.67, -41.79, -42.83, -45.07, -45.31, -45.95, -46.27, -47.15, -47.23, -47.63, -49.19, -49.72, -50.03, -50.08, -50.51, -51.64, -52.07, -52.28, -52.52, -53.36, -55.19, -55.28, -56.24, -56.36, -56.92, -57.2, -58.4, -58.84, -59.51, -59.96, -60.76, -60.84, -60.88, -62.56, -62.56, -64.49, -65.48, -65.52, -65.69, -65.93, -66.0, -67.49, -67.88, -68.0, -68.96, -69.08, -69.77, -70.52, -71.97, -72.05, -72.53, -72.96, -74.0